<a href="https://colab.research.google.com/github/Evaggelou/Arvato-Project-/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
# load pickle objects
X_train = pickle.load(open("/content/drive/MyDrive/Udacity/Arvato/X.pickle", "rb"))
y_train = pickle.load(open("/content/drive/MyDrive/Udacity/Arvato/y.pickle", "rb"))

In [4]:
!pip install catboost

     |████████████████████████████████| 67.3MB 93kB/s 


In [5]:
import catboost

def catboost_classifier(depth, l2_leaf_reg, num_boost_round):
    params = {
        "loss_function": "Logloss",
        "eval_metric" : "AUC", 
        "depth" : int(depth),
        "l2_leaf_reg" : int(l2_leaf_reg),
        "learning_rate" : 0.01,
        "random_state" : 42,
        "logging_level" : "Silent",
        "thread_count": 24,
        "num_boost_round": int(num_boost_round)
    }
    train_data = catboost.Pool(data=X_train, label=y_train)
    cv_result = catboost.cv(train_data,
                       params,
                       early_stopping_rounds=100,
                       stratified=True,
                       nfold=5)
    return cv_result['test-AUC-mean'].iloc[-1]

In [6]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=0caef80bdbf9b1c807a9fb5c297132eaa3d3fde160b0ad647f327aa28f75c88c
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [8]:
from bayes_opt import BayesianOptimization
catboostBO = BayesianOptimization(catboost_classifier, {
                                                'depth': (1, 4),
                                                'l2_leaf_reg': (2, 30),
                                                'num_boost_round': (100, 1000),
                                                })

catboostBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   |   depth   | l2_lea... | num_bo... |
-------------------------------------------------------------
|  1        |  0.7747   |  3.139    |  26.17    |  948.0    |
|  2        |  0.7689   |  2.718    |  17.05    |  948.9    |
|  3        |  0.6517   |  2.259    |  29.95    |  100.1    |
|  4        |  0.7559   |  1.398    |  29.99    |  639.3    |
|  5        |  0.7692   |  2.836    |  29.43    |  999.6    |
|  6        |  0.7741   |  3.973    |  29.91    |  835.9    |
|  7        |  0.7579   |  1.044    |  29.95    |  887.9    |
|  8        |  0.7747   |  3.201    |  26.04    |  944.3    |
|  9        |  0.7713   |  3.263    |  2.722    |  339.5    |
|  10       |  0.7733   |  3.981    |  29.89    |  436.7    |
|  11       |  0.7712   |  3.647    |  29.5     |  277.9    |
|  12       |  0.7733   |  3.766    |  2.779    |  729.6    |


In [9]:
print(catboostBO.max)

{'target': 0.7747329093240714, 'params': {'depth': 3.1386012415294946, 'l2_leaf_reg': 26.169421579838826, 'num_boost_round': 947.9752915393171}}
